# Serve Mistral-7B on SageMaker and Inferentia using the LMI container.


In this notebook, we deploy the [Mistral-7B](https://huggingface.co/mistralai/Mistral-7B-v0.1) model on SageMaker by leveraging the [SageMaker Large Model Inference Container](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#large-model-inference-containers).

This notebook explains how to deploy model optimized for latency and throughput. The tuning guide is available [LLM Tuning Guide](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#large-model-inference-containers).

## Install, import the required libraries, and set some variables

In [ ]:
%pip install sagemaker boto3 huggingface_hub awscli --upgrade  --quiet

In [ ]:
import json
import boto3
import sagemaker
from sagemaker import Model, image_uris, serializers, deserializers

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

### Select the appropriate configuration parameters and container

To optimize the deployment of Large Language Models (LLMs); one needs to choose the appropriate model partitioning framework, optimal batching technique, batching size, tensor parallelism degree, etc. The choice of a particular configuration depends on the use case.

Hence, based on the use case, you need to:
1. Set the configuration parameters for the container.
2. Select the appropriate container image to be used for inference.

### Set the configuration parameters using environment variables
1. `SERVING_LOAD_MODELS` - specifies the engine that will be used for this workload. In this case we'll be hosting a model using the **Python** engine.

2. `OPTION_MODEL_ID`: Set this to the URI of the Amazon S3 bucket that contains the model. When this is set, the container leverages [s5cmd](https://github.com/peak/s5cmd) to download the model from s3. This enables faster deployments by utilizing optimized approach within the DJL inference container to transfer the model from S3 into the hosting instance.
If you want to download the model from huggingface.co, you can set `OPTION_MODEL_ID` to the model id of a pre-trained model hosted inside a model repository on huggingface.co (https://huggingface.co/models). The container uses this model id to download the corresponding model repository on huggingface.co.
- If the model weights are in the standard HuggingFace format, runtime compilation is performed
- You can also provide compiled models and split-models. The source of the weights can be huggingface model hub, S3 or a local path. 

3. `OPTION_TENSOR_PARALLEL_DEGREE`: Set to the number of neuron cores over which the model needs to be partitioned. This parameter also controls the number of workers per model which will be started up when DJL serving runs. In this example we use the `ml.inf2.8xlarge` instance that has 1 accelerators with 2 neuron cores, hence it is set to 2. Refer the [docs](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/arch/neuron-hardware/inf2-arch.html) for more details about the inferentia architecture.

4. `OPTION_ROLLING_BATCH`: This parameter enables the use of a particular batching technique for continuous or iteration level batching to enable merging multiple concurrent requests that arrive at different times for inference. In this scenario we set it to `auto` and the framework uses a custom rolling batch mechanism.

5. `OPTION_MAX_ROLLING_BATCH_SIZE`: The maximum number of concurrent requests to be used in a batch by the model server for inference. Clients can still send more requests to the endpoint, they will be queued.

6. `OPTION_N_POSITIONS`: This parameter controls the **Total sequence length, i.e. input sequence length + output sequence length** that you want the model to process during inference. If you are providing weights in standard huggingface format and performing runtime compilation, set this to the sequence length you would observe when using the endpoint.

For more information on the available options, please refer to the [DJL Serving - SageMaker Large Model Inference Configurations](https://github.com/deepjavalibrary/djl-serving/blob/master/serving/docs/lmi/configurations_large_model_inference_containers.md) and [NeuronX Configurations](https://github.com/deepjavalibrary/djl-serving/blob/master/serving/docs/lmi/user_guides/tnx_user_guide.md#advanced-transformers-neuronx-configurations)


### Select the relevant Large Model Inference container
SageMaker offers optimized [large model inference containers](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#large-model-inference-containers) that contains different frameworks for model parallelism enabling inference of LLMs on multiple accelerators.

In this scenario, we leverage the `NeuronX` container.

In [ ]:
neuron_image_uri = image_uris.retrieve(
    framework="djl-neuronx", region=sess.boto_session.region_name, version="0.26.0"
)

env_generation = {
    "HUGGINGFACE_HUB_CACHE": "/tmp",
    "TRANSFORMERS_CACHE": "/tmp",
    "SERVING_LOAD_MODELS": "test::Python=/opt/ml/model",
    "OPTION_MODEL_ID": "mistralai/Mistral-7B-v0.1",
    "OPTION_TENSOR_PARALLEL_DEGREE": "2",  # Inf2.8xlarge has 1 Inferentia2 accelerator (with two NeuronCore-v2 cores).
    "OPTION_ROLLING_BATCH": "auto",
    "OPTION_MAX_ROLLING_BATCH_SIZE": "4",
    "OPTION_N_POSITIONS": "512",
}

In [ ]:
print(neuron_image_uri)

In [ ]:
# - Select the appropriate environment variable which will tune the deployment server.
env = env_generation  # use this in case it is 'generation' task

# - now we select the appropriate container
inference_image_uri = neuron_image_uri  # use this in case it is 'generation' task


print(f"Environment variables are ---- > {env}")
print(f"Image going to be used is ---- > {inference_image_uri}")

### To create the endpoint the steps are:
- Create the Model using the inference image container

- Create the endpoint config using the following key parameters

In this notebook we leverage the boto3 SDK. You can also use the [SageMaker SDK](https://sagemaker.readthedocs.io/en/stable/).

In [ ]:
instance_type = "ml.inf2.8xlarge"
endpoint_name = sagemaker.utils.name_from_base("Mistral-7b")

### Create the Model
Leverage the `inference_image_uri` to create a model object. We will leverage the Least routing algorithim -- [Least Routing Algorithim](https://aws.amazon.com/blogs/machine-learning/minimize-real-time-inference-latency-by-using-amazon-sagemaker-routing-strategies/). This innovation from sagemnaker has shown to reduce latency by 10% or more when we have multiple instances configured to serve the endpoints

In [ ]:
model_name = sagemaker.utils.name_from_base("lmi-Mistral-7b")
print(model_name)

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "Environment": env,
    },
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

### Create an endpoint config
Create an endpoint configuration using the appropriate instance type. Set the `ContainerStartupHealthCheckTimeoutInSeconds` to account for the time taken to download the LLM weights from S3 or the model hub; and the time taken to load the model on the accelerators.

In [ ]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": instance_type,
            "InitialInstanceCount": 1,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 2400
        },
    ],
)
endpoint_config_response

### Create an endpoint using the model and endpoint config

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

#### This step can take ~15 mins or longer

In [ ]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

### Invoke the endpoint with a sample prompt

In [ ]:
# use this for Chatbot or QA or open ended generation task
prompt = "Amazon.com is the best"
params = {"max_new_tokens": 512, "do_sample": False}

In [ ]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps({"inputs": prompt, "parameters": params}),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

## Clean up the environment

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

#### Resource:
- [Deep Learning containers for large model inference](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-dlc.html)
- [Deep Java Library for Large Model Inference](https://docs.djl.ai/docs/serving/serving/docs/large_model_inference.html)